### Polecenie od klienta:
*"Mamy co prawda dodatkowe benefity dla naszych najlepszych klientów, ale może dałoby
się ustalić kto potencjalnie jest skłonny wydawać u nas więcej?”*

### Zadanie biznesowe
Sugerowanie klientów, którzy mogą wrócić do serwisu.

### Zadanie modelowania
Model regresyjny, szacujący prawdopodobieństwo powrotu klienta do serwisu.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
iteration_path = "iteration_3/"
deliveries_path = "../data/" + iteration_path + "raw/deliveries.jsonl"
products_path = "../data/" + iteration_path + "raw/products.jsonl"
sessions_path = "../data/" + iteration_path + "raw/sessions.jsonl"
users_path = "../data/" + iteration_path + "raw/users.jsonl"

In [3]:
deliveries_data = pd.read_json(deliveries_path, lines=True)
products_data = pd.read_json(products_path, lines=True)
sessions_data = pd.read_json(sessions_path, lines=True)
users_data = pd.read_json(users_path, lines=True)

In [30]:
users_data

,user_id,name,city,street
0,102,Arkadiusz Hejna,Wrocław,ul. Jana 32
1,103,Oskar Skotarczak,Warszawa,pl. Wrzosowa 595
2,104,Maksymilian Fik,Gdynia,al. Podwale 62
3,105,Fryderyk Hajdukiewicz,Gdynia,al. Narcyzowa 81/19
4,106,Marcelina Kinder,Wrocław,pl. Irysowa 472
...,...,...,...,...
195,297,Olga Myrcha,Kraków,plac Listopada 83/33
196,298,Nikodem Świgoń,Warszawa,ul. Złota 61/12
197,299,Gaja Wojtarowicz,Gdynia,ul. Baczynskiego 97
198,300,Inga Przewoźny,Radom,ulica Wysoka 33


In [48]:
sessions_data

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id,timestamp_week,timestamp_quarter,timestamp_date,timestamp_month
5462,1615,2021-01-08 17:29:37,139,1032,VIEW_PRODUCT,10,NaN,1,1,2021-01-08,1
5463,1615,2021-01-08 17:32:51,139,1033,VIEW_PRODUCT,10,NaN,1,1,2021-01-08,1
5464,1615,2021-01-08 17:35:32,139,1033,BUY_PRODUCT,10,20833.0,1,1,2021-01-08,1
5544,1635,2021-01-10 05:56:20,139,1292,VIEW_PRODUCT,20,NaN,1,1,2021-01-10,1
5545,1635,2021-01-10 05:57:54,139,1291,VIEW_PRODUCT,20,NaN,1,1,2021-01-10,1
...,...,...,...,...,...,...,...,...,...,...,...
12596,3603,2021-12-10 11:21:21,200,1041,VIEW_PRODUCT,20,NaN,49,4,2021-12-10,12
12597,3603,2021-12-10 11:23:13,200,1047,VIEW_PRODUCT,20,NaN,49,4,2021-12-10,12
12598,3603,2021-12-10 11:26:10,200,1040,VIEW_PRODUCT,20,NaN,49,4,2021-12-10,12
12599,3603,2021-12-10 11:26:46,200,1040,BUY_PRODUCT,20,21867.0,49,4,2021-12-10,12


In [31]:
sessions_data = sessions_data.sort_values(by=['timestamp'])
sessions_data['timestamp_week'] = sessions_data['timestamp'].apply(lambda x: x.week)
sessions_data['timestamp_month'] = sessions_data['timestamp'].apply(lambda x: x.month)
sessions_data['timestamp_quarter'] = sessions_data['timestamp'].apply(lambda x: x.quarter)
sessions_data['timestamp_date'] = sessions_data['timestamp'].apply(lambda x: x.date())

Podział na zbiór testowy i treningowy

In [32]:
sample_month_data = sessions_data[sessions_data.timestamp_month == 9]
train_data = sessions_data[sessions_data.timestamp_quarter < 4]
test_data = sessions_data[sessions_data.timestamp_quarter == 4]

Przykład join-a

In [33]:
pd.merge(train_data, products_data, on="product_id").sort_values(by=['timestamp'])

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id,timestamp_week,timestamp_quarter,timestamp_date,timestamp_month,product_name,category_path,price
0,1615,2021-01-08 17:29:37,139,1032,VIEW_PRODUCT,10,NaN,1,1,2021-01-08,1,LCD Iiyama E2280WSD,Komputery;Monitory;Monitory LCD,688.78
197,1615,2021-01-08 17:32:51,139,1033,VIEW_PRODUCT,10,NaN,1,1,2021-01-08,1,LCD Iiyama T1932MSC,Komputery;Monitory;Monitory LCD,3029.00
198,1615,2021-01-08 17:35:32,139,1033,BUY_PRODUCT,10,20833.0,1,1,2021-01-08,1,LCD Iiyama T1932MSC,Komputery;Monitory;Monitory LCD,3029.00
383,1635,2021-01-10 05:56:20,139,1292,VIEW_PRODUCT,20,NaN,1,1,2021-01-10,1,Philips SDV8622,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,189.00
574,1635,2021-01-10 05:57:54,139,1291,VIEW_PRODUCT,20,NaN,1,1,2021-01-10,1,Philips SDV6224,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,168.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8942,336,2021-09-30 23:03:20,106,1030,VIEW_PRODUCT,0,NaN,39,3,2021-09-30,9,LCD Iiyama B2280WSD,Komputery;Monitory;Monitory LCD,739.00
8943,336,2021-09-30 23:05:46,106,1030,BUY_PRODUCT,0,20116.0,39,3,2021-09-30,9,LCD Iiyama B2280WSD,Komputery;Monitory;Monitory LCD,739.00
11546,6769,2021-09-30 23:42:01,298,1276,VIEW_PRODUCT,0,NaN,39,3,2021-09-30,9,Apple iPad mini 64GB,Komputery;Tablety i akcesoria;Tablety,1816.97
13730,6769,2021-09-30 23:45:23,298,1277,VIEW_PRODUCT,0,NaN,39,3,2021-09-30,9,Apple iPad mini 64GB 4G,Komputery;Tablety i akcesoria;Tablety,2317.02


In [34]:
def get_user_id_from_session(session):
    sample_user_id = session['user_id'].iloc[0]
    for user_id in session['user_id']:
        if sample_user_id != user_id:
            raise Exception("How it is even possible")
    return sample_user_id


In [35]:
def check_if_user_bought_something(session):
    for event_type in session['event_type']:
        if event_type == 'BUY_PRODUCT':
            return True
    return False

In [36]:
def extract_session(session_id):
    session = sessions_data[sessions_data['session_id'] == session_id]
    d = {'session_id': session_id, 'beginning': [min(session['timestamp'])], 'end': [max(session['timestamp'])], 'user_id' : get_user_id_from_session(session),
         'bought_product': check_if_user_bought_something(session)}
    df = pd.DataFrame(data=d)
    return df.set_index('session_id')

In [37]:
def extract_session_data(sessions_data):
    sessions = []
    for session_id in sessions_data['session_id'].unique():
        sessions.append(extract_session(session_id))
    extracted_session_data = pd.concat(sessions)
    return extracted_session_data

In [55]:
extracted_session_data = extract_session_data(sample_month_data)

In [46]:
extracted_session_data[extracted_session_data.bought_product == True]

,beginning,end,user_id,bought_product
session_id,,,,
1615,2021-01-08 17:29:37,2021-01-08 17:35:32,139,True
1635,2021-01-10 05:56:20,2021-01-10 06:23:24,139,True
369,2021-01-11 00:25:44,2021-01-11 00:28:09,108,True
1791,2021-01-12 08:48:51,2021-01-12 08:51:03,143,True
1692,2021-01-13 05:33:33,2021-01-13 05:40:02,140,True
...,...,...,...,...
6641,2021-12-10 05:51:38,2021-12-10 05:53:39,295,True
3744,2021-12-10 07:03:25,2021-12-10 07:09:48,204,True
1244,2021-12-10 09:15:24,2021-12-10 09:19:52,128,True


In [50]:
extracted_session_data = extract_session_data(sessions_data)

In [39]:
def find_returned_users(extracted_sessions_data):
    user_counts = extracted_sessions_data['user_id'].value_counts()
    return user_counts[user_counts>=2].index

In [53]:
def find_never_returned_users(extracted_sessions_data):
    user_counts = extracted_sessions_data['user_id'].value_counts()
    return user_counts[user_counts<2].index

In [59]:
extracted_session_data

,beginning,end,user_id,bought_product
session_id,,,,
2246,2021-09-01 02:00:24,2021-09-01 02:04:10,158,True
198,2021-09-01 02:25:19,2021-09-01 02:33:16,103,True
267,2021-09-01 02:52:11,2021-09-01 02:52:11,104,False
171,2021-09-01 03:16:43,2021-09-01 03:19:30,103,True
1743,2021-09-01 03:33:57,2021-09-01 03:35:08,140,True
...,...,...,...,...
572,2021-09-30 20:40:48,2021-09-30 21:01:49,113,True
2502,2021-09-30 21:28:55,2021-09-30 21:28:55,165,False
2595,2021-09-30 21:45:54,2021-09-30 21:58:43,168,False


In [58]:
len(find_never_returned_users(extracted_session_data)) + len(find_returned_users(extracted_session_data))

181

# Do poprawy
 - ustalone kryteria sukcesu -> pole pod krzywą ROC, zamiast wartość krzywej
 - brak sprawdzenia typów atrybutów/zakresy wartości -> histogramy + mądre zakresy
 - brak weryfikacji, czy dane wydają się nadawać do modelowania (czy zmienne wejściowe coś mówią o zmiennej wyjściowej) -> dobre pytanie, właśnie to badam